In [1]:
import numpy as np
import matplotlib.pyplot as plt

import scarlet
from astropy.wcs import WCS
import astropy.io.fits as fits
from astropy.nddata import Cutout2D

import warnings
warnings.simplefilter("ignore")

In [2]:
#paths
path = '/Users/rahim/Desktop/repo/AGN_Deblending/Extract/data/HSC_Images/'
path_hst = '/Users/rahim/Desktop/repo/AGN_Deblending/Extract/data/acs_mosaic_2.0/'

#Hst data
hdu_HST= fits.open(path_hst + '10_150.1797581_2.1103119_acs_I_mosaic_30mas_sci.fits')

#HSC data
hdu_HSC_G= fits.open(path + '10-cutout-HSC-G-9813-pdr2_dud.fits')
hdu_HSC_R= fits.open(path + '10-cutout-HSC-R-9813-pdr2_dud.fits')
hdu_HSC_I= fits.open(path + '10-cutout-HSC-I-9813-pdr2_dud.fits')
hdu_HSC_Z= fits.open(path + '10-cutout-HSC-Z-9813-pdr2_dud.fits')
hdu_HSC_Y= fits.open(path + '10-cutout-HSC-Y-9813-pdr2_dud.fits')

In [3]:
def mk_cut(image, wcs, coord, size = 8.35):
    """ Makes a cutout of an image and its wcs around a coordinate
    
    Parameters
    ----------
    data: numpy array
        image
    wcs: WCS
        wcs of the image
    coord: tuple
        coordinates around which to cut (degrees)
    size: float
        width of the cutout in arcseconds. default is 8.35 (~50 HSC pixels)
    """
    try:
        model_affine = wcs.wcs.pc
    except AttributeError:
        model_affine = wcs.wcs.cd
    #computes the pixel scale of the observation in arcseconds
    pix_scale = np.sqrt(np.abs(model_affine[0, 0])
        * np.abs(model_affine[1, 1] - model_affine[0, 1] * model_affine[1, 0]))
    #Center of the cut in (ra, dec)
    ra, dec = coord
    #Converts center in pixels. To make a cut around the center of an image, simply use the index of the central pixel instead.
    if np.size(wcs.array_shape) == 2:
        y, x = wcs.all_world2pix(ra, dec, 0, ra_dec_order=True)
    elif np.size(wcs.array_shape) == 3:
        y, x, _ = wcs.all_world2pix(ra, dec, 0, 0, ra_dec_order=True)
    #Size of the cut in pixels.
    radius = size / 3600 / pix_scale
    #Make the cut  in the image AND the WCS
    cut = Cutout2D(image, (y, x), (radius, radius), wcs = wcs)

    return cut

def mk_psf(coord):
    """ makes single band psfs into cubes of psfs
    """
    path_psf = '/Users/rahim/Desktop/repo/AGN_Deblending/Extract/data/HSC_PSF/'
    
    psf = fits.open(path_psf + f'10-psf-calexp-pdr2_dud-HSC-I-9813-4,3-{coord[0]:.5f}-{coord[1]:.5f}.fits')[0].data
    psfs = np.zeros((5, np.shape(psf)[-2], np.shape(psf)[-1]))
    
    for i,filter in enumerate(('G', 'R', 'I', 'Z', 'Y')):
        filename = path_psf + f'10-psf-calexp-pdr2_dud-HSC-{filter}-9813-4,3-{coord[0]:.5f}-{coord[1]:.5f}.fits'
        psf = fits.open(filename)[0].data
        psfs[i] = psf
    name = f'psf-calexp-pdr2_dud-HSC-9813-4,3-{coord[0]:.5f}-{coord[1]:.5f}.fits'
    return np.array(psfs), name
    print(filename)


In [5]:
def mk_hst_hsc_cuts(coord, size = 9):
    """ Saves the cuts in fits files
    """
    cut_hst = mk_cut(hdu_HST[0].data, WCS(hdu_HST[0].header), coord, size = size)
    cut_hsc_g = mk_cut(hdu_HSC_G[1].data, WCS(hdu_HSC_G[1].header), coord, size = 9)
    cut_hsc_r = mk_cut(hdu_HSC_R[1].data, WCS(hdu_HSC_R[1].header), coord, size = 9)
    cut_hsc_i = mk_cut(hdu_HSC_I[1].data, WCS(hdu_HSC_I[1].header), coord, size = 9)
    cut_hsc_z = mk_cut(hdu_HSC_Z[1].data, WCS(hdu_HSC_Z[1].header), coord, size = 9)
    cut_hsc_y = mk_cut(hdu_HSC_Y[1].data, WCS(hdu_HSC_Y[1].header), coord, size = 9)
    
    cuts = (cut_hsc_g, cut_hsc_r, cut_hsc_i, cut_hsc_z, cut_hsc_y)
    n1,n2 = cut_hsc_g.shape
    data_hsc = np.zeros((5,n1,n2))
    for i in range(len(cuts)):
        data_hsc[i] = cuts[i].data / np.sum(cuts[i].data) * np.size(cuts[i].data)

    cut_hst.data /= np.sum(cut_hst.data)/np.size(cut_hst.data)
    #WCS is stored in the header (hdr) of the fits 
    #Here I arbitrarily use the header of the i-band to make the cube header
    hdr_hsc = cut_hsc_i.wcs.to_header()
    hdr_hst = cut_hst.wcs.to_header()

    plt.subplot(121)
    plt.imshow(np.log10(cut_hst.data), cmap = 'gist_stern')
    plt.subplot(122)
    plt.imshow(cut_hsc_i.data, cmap = 'gist_stern')
    plt.show()
    
    #The header argument is how you save the wcs 
    hdus = fits.PrimaryHDU(np.array(data_hsc), header = hdr_hsc)
    #print(hdus.shape, hdus.dtype, hdus.dtype.name, hdus.dtype.type)
    lists = fits.HDUList([hdus])
#     lists.writeto(F'hsc_ra={coord[0]:.5f}_dec={coord[1]:.5f}.fits', clobber=True)
    
    hdus = fits.PrimaryHDU(cut_hst.data, header = hdr_hst)
    lists = fits.HDUList([hdus])
#     lists.writeto(F'hst_ra={coord[0]:.5f}_dec={coord[1]:.5f}.fits', clobber=True)
    
    psf, name = mk_psf(coord)
    
    hdus = fits.PrimaryHDU(psf)
    lists = fits.HDUList([hdus])
#     lists.writeto(f'psf_hsc_ra={coord[0]:.5f}_dec={coord[1]:.5f}.fits', clobber=True)
#     pass

In [7]:
# mk_hst_hsc_cuts((150.2311511, 2.0725000), size = 8)
#mk_hst_hsc_cuts((150.2357461, 2.0736144))
#mk_hst_hsc_cuts((150.2407120, 2.0651355))
#mk_hst_hsc_cuts((150.2537000, 2.0480000))
#mk_hst_hsc_cuts((150.3054000, 2.0823254))
#mk_hst_hsc_cuts((150.3305180, 2.0707583))
# mk_hst_hsc_cuts((149.4978988, 2.0766533))
mk_hst_hsc_cuts((150.1947519, 2.0678959))

NoOverlapError: Arrays do not overlap.